##Индивидуальное задание
Написать программу на языке программирования Python для решения
поставленной задачи (в репозитории должны присутствовать настройки требуемых
пакетов для выбранного менеджера пакетов). Приложение должно использовать
интерфейс командной строки (модуль argparse ) или графический интерфейс
пользователя (модули tkinter , PySide2 , Kivy и т. д.). При работе с датой и
временем использовать пакет datetime . Организовать чтение и сохранение
данных из/в формат Parquet. Выполнить валидацию сохраненных данных с
помощью сторонних приложений для работы с форматом Parquet, например
https://github.com/mukunku/ParquetViewer. Организовать также удаление данных по
одной из колонок DataFrame на выбор обучающихся. Номер варианта
определяется по согласованию с преподавателем.Использовать DataFrame , содержащий следующие колонки: название
начального пункта маршрута; название конечного пункта маршрута; номер
маршрута. Написать программу, выполняющую следующие действия: ввод с
клавиатуры данных и добавление строк в DataFrame ; записи должны быть
упорядочены по номерам маршрутов; вывод на экран информации о
маршруте, номер которого введен с клавиатуры; если таких маршрутов нет,
выдать на дисплей соответствующее сообщение.

In [9]:
!pip install pyarrow ipywidgets


In [10]:
import pandas as pd
import os
from datetime import datetime
from IPython.display import display, clear_output
import ipywidgets as widgets

class RouteManager:
    def __init__(self):
        self.df = pd.DataFrame(columns=['start_point', 'end_point', 'route_number', 'created_at'])

    def add_route(self, start: str, end: str, number: int):
        """Добавление нового маршрута с автоматической сортировкой"""
        new_route = pd.DataFrame({
            'start_point': [start],
            'end_point': [end],
            'route_number': [number],
            'created_at': [datetime.now()]
        })
        self.df = pd.concat([self.df, new_route], ignore_index=True)
        self.df = self.df.sort_values(by='route_number').reset_index(drop=True)
        return f"✅ Маршрут №{number} добавлен: {start} → {end}"

    def find_route(self, number: int):
        """Поиск маршрута по номеру"""
        result = self.df[self.df['route_number'] == number]
        return result

    def delete_by_column(self, column: str, value: str):
        """Удаление записей по значению в колонке"""
        if column in self.df.columns:
            before = len(self.df)
            self.df = self.df[self.df[column] != value]
            return f"🗑️ Удалено {before - len(self.df)} записей где {column} = {value}"
        return "❌ Ошибка: неверное название колонки"

    def save_to_parquet(self, filename: str):
        """Сохранение данных в Parquet-формат"""
        self.df.to_parquet(filename, engine='pyarrow')
        return f"💾 Данные сохранены в {filename}"

    def load_from_parquet(self, filename: str):
        """Загрузка данных из Parquet-файла"""
        if os.path.exists(filename):
            self.df = pd.read_parquet(filename, engine='pyarrow')
            self.df = self.df.sort_values(by='route_number').reset_index(drop=True)
            return f"📂 Данные загружены из {filename}\nВсего маршрутов: {len(self.df)}"
        return f"❌ Файл {filename} не найден"

    def show_all_routes(self):
        """Показать все маршруты"""
        if self.df.empty:
            return "🚫 Нет данных о маршрутах"
        return self.df[['start_point', 'end_point', 'route_number']]

def create_ui():
    """Создание пользовательского интерфейса"""
    manager = RouteManager()
    output = widgets.Output()

    # Виджеты для добавления маршрута
    start_input = widgets.Text(placeholder='Москва', description='Откуда:')
    end_input = widgets.Text(placeholder='Санкт-Петербург', description='Куда:')
    number_input = widgets.IntText(description='Номер:', value=1)
    add_button = widgets.Button(description='Добавить', button_style='success')

    # Виджеты для поиска маршрута
    search_input = widgets.IntText(description='Поиск:', value=1)
    search_button = widgets.Button(description='Найти', button_style='info')

    # Виджеты для удаления маршрутов
    column_dropdown = widgets.Dropdown(
        options=['Начальный пункт', 'Конечный пункт', 'Номер маршрута'],
        value='Номер маршрута',
        description='Удалить по:'
    )
    value_input = widgets.Text(description='Значение:', value='')
    delete_button = widgets.Button(description='Удалить', button_style='danger')

    # Виджеты для работы с файлами
    filename_input = widgets.Text(value='routes.parquet', description='Файл:')
    save_button = widgets.Button(description='Сохранить', button_style='primary')
    load_button = widgets.Button(description='Загрузить', button_style='primary')

    # Обработчики событий
    def on_add_button_clicked(b):
        with output:
            clear_output()
            try:
                result = manager.add_route(
                    start_input.value,
                    end_input.value,
                    number_input.value
                )
                print(result)
                display(manager.show_all_routes())
            except Exception as e:
                print(f"⛔ Ошибка: {str(e)}")

    def on_search_button_clicked(b):
        with output:
            clear_output()
            try:
                routes = manager.find_route(search_input.value)
                if not routes.empty:
                    print(f"🔍 Найденные маршруты №{search_input.value}:")
                    display(routes[['start_point', 'end_point', 'route_number']])
                else:
                    print(f"🚫 Маршрутов с номером {search_input.value} не найдено")
            except Exception as e:
                print(f"⛔ Ошибка: {str(e)}")

    def on_delete_button_clicked(b):
        with output:
            clear_output()
            try:
                # Преобразование русских названий в английские
                column_map = {
                    'Начальный пункт': 'start_point',
                    'Конечный пункт': 'end_point',
                    'Номер маршрута': 'route_number'
                }
                column = column_map[column_dropdown.value]

                # Преобразование значения для номера маршрута
                if column == 'route_number':
                    value = int(value_input.value)
                else:
                    value = value_input.value

                result = manager.delete_by_column(column, value)
                print(result)
                display(manager.show_all_routes())
            except Exception as e:
                print(f"⛔ Ошибка: {str(e)}")

    def on_save_button_clicked(b):
        with output:
            clear_output()
            try:
                print(manager.save_to_parquet(filename_input.value))
            except Exception as e:
                print(f"⛔ Ошибка: {str(e)}")

    def on_load_button_clicked(b):
        with output:
            clear_output()
            try:
                result = manager.load_from_parquet(filename_input.value)
                print(result)
                display(manager.show_all_routes())
            except Exception as e:
                print(f"⛔ Ошибка: {str(e)}")
    # Привязка обработчиков
    add_button.on_click(on_add_button_clicked)
    search_button.on_click(on_search_button_clicked)
    delete_button.on_click(on_delete_button_clicked)
    save_button.on_click(on_save_button_clicked)
    load_button.on_click(on_load_button_clicked)
    # Группировка виджетов
    add_box = widgets.VBox([
        widgets.HTML("<h3>Добавить маршрут</h3>"),
        widgets.HBox([start_input, end_input, number_input, add_button])
    ])
    search_box = widgets.VBox([
        widgets.HTML("<h3>Поиск маршрута</h3>"),
        widgets.HBox([search_input, search_button])
    ])

    delete_box = widgets.VBox([
        widgets.HTML("<h3>Удалить маршруты</h3>"),
        widgets.HBox([column_dropdown, value_input, delete_button])
    ])

    file_box = widgets.VBox([
        widgets.HTML("<h3>Работа с файлами</h3>"),
        widgets.HBox([filename_input, save_button, load_button])
    ])
    # Отображение интерфейса
    display(widgets.VBox([
        widgets.HTML("<h2 style='color: #1a73e8;'>🚌 Управление маршрутами</h2>"),
        add_box,
        search_box,
        delete_box,
        file_box,
        widgets.HTML("<h3>Результат:</h3>"),
        output
    ]))
# Запуск интерфейса
create_ui()